Removing 0 from DEM 

In [ ]:
import rasterio
import numpy as np

def process_geotiff(input_file, output_file):
    # Open the input TIFF file
    with rasterio.open(input_file) as src:
        # Read the data from the first band
        data = src.read(1)
        
        # Set negative and zero values to NaN
        data[data == 0] = np.nan
        
        # Update the metadata to handle NaN values correctly
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,  # Ensure data type can handle NaN
            nodata=np.nan            # Define nodata value
        )
        
        # Write the modified data to a new TIFF file
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(data, 1)

# Define input and output file paths
input_file = 'UK_DEM_merged.tif'
output_file = 'UK_DEM2.tif'

# Process the TIFF file
process_geotiff(input_file, output_file)

Making Points out of DEM

In [4]:
import rasterio
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm

# Path to your GeoTIFF file
input_file = 'UK_DEM2.tif'

# Open the GeoTIFF file
with rasterio.open(input_file) as src:
    # Read the entire raster data into memory
    data = src.read(1)
    
    # Get the nodata value
    nodata_value = src.nodata

    # Create a mask for valid data points (including zero values)
    mask = (data != nodata_value) & ~np.isnan(data)
    
    # Get the row and column indices of valid data points
    row_indices, col_indices = np.where(mask)
    
    # Get the pixel values of the valid data points
    values = data[mask]
    
    # Initialize lists to store coordinates and values
    lons, lats = [], []
    
    # Iterate with progress tracking
    for row, col in tqdm(zip(row_indices, col_indices), total=len(row_indices), desc="Processing"):
        lon, lat = src.xy(row, col)
        lons.append(lon)
        lats.append(lat)

# Create a DataFrame from the valid points
df = pd.DataFrame({
    'longitude': lons,
    'latitude': lats,
    'value': values
})

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs=src.crs)

# Save the GeoDataFrame as a shapefile
output_shapefile = 'UK_DEM.shp'
gdf.to_file(output_shapefile)

print(f"Shapefile saved successfully: {output_shapefile}")


Processing: 100%|██████████| 1346519/1346519 [00:49<00:00, 26943.62it/s]


Shapefile saved successfully: UK_DEM.shp


Current Idea make a take the bounds of 12 km square, and if lat and long of 30m fall in the bounds of 12km square then we delete those points. 

Removing 0 from temprature 

In [ ]:
import rasterio
import numpy as np

def process_geotiff(input_file, output_file):
    # Open the input TIFF file
    with rasterio.open(input_file) as src:
        # Read the data from the first band
        data = src.read(1)
        
        # Set negative and zero values to NaN
        data[data == -99] = np.nan
        
        # Update the metadata to handle NaN values correctly
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,  # Ensure data type can handle NaN
            nodata=np.nan            # Define nodata value
        )
        
        # Write the modified data to a new TIFF file
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(data, 1)

# Define input and output file paths
input_file = 'UK_Avg_temp_Band1.tif'
output_file = 'UK_temp2.tif'

# Process the TIFF file
process_geotiff(input_file, output_file)

Making temprature data polygons 

In [3]:
import rasterio
from shapely.geometry import Point, box
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm

# Path to your GeoTIFF file
input_file = 'UK_temp2.tif'

# Open the GeoTIFF file
with rasterio.open(input_file) as src:
    # Read the entire raster data into memory
    data = src.read(1)
    
    # Get the nodata value
    nodata_value = src.nodata

    # Create a mask for valid data points (including zero values)
    mask = (data != nodata_value) & ~np.isnan(data)
    
    # Get the row and column indices of valid data points
    row_indices, col_indices = np.where(mask)
    
    # Get the pixel values of the valid data points
    values = data[mask]
    
    # Initialize lists to store coordinates, values, and bounds
    lons, lats, x_mins, y_mins, x_maxs, y_maxs = [], [], [], [], [], []
    
    # Get pixel size (resolution)
    pixel_size_x, pixel_size_y = src.res
    
    # Iterate with progress tracking
    for row, col in tqdm(zip(row_indices, col_indices), total=len(row_indices), desc="Processing"):
        lon, lat = src.xy(row, col)
        lons.append(lon)
        lats.append(lat)
        x_min, y_min = lon - pixel_size_x / 2, lat - pixel_size_y / 2
        x_max, y_max = lon + pixel_size_x / 2, lat + pixel_size_y / 2
        x_mins.append(x_min)
        y_mins.append(y_min)
        x_maxs.append(x_max)
        y_maxs.append(y_max)

# Create a DataFrame from the valid points
df = pd.DataFrame({
    'longitude': lons,
    'latitude': lats,
    'value': values,
    'x_min': x_mins,
    'y_min': y_mins,
    'x_max': x_maxs,
    'y_max': y_maxs
})

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry=[box(x_min, y_min, x_max, y_max) for x_min, y_min, x_max, y_max in zip(df.x_min, df.y_min, df.x_max, df.y_max)], crs=src.crs)

# Save the GeoDataFrame as a shapefile
output_shapefile = 'UK_temp2.shp'
gdf.to_file(output_shapefile)

print(f"Shapefile saved successfully: {output_shapefile}")


Processing: 100%|██████████| 2498/2498 [00:00<00:00, 21018.68it/s]


Shapefile saved successfully: UK_temp2.shp


### Need to really understand this sorting algorithm 

In [17]:
import geopandas as gpd
from shapely.geometry import Point
import rtree
import pandas as pd
from tqdm import tqdm

# Load the shapefile with polygons (bounding boxes)
polygons_gdf = gpd.read_file('UK_temp2.shp')

# Load the shapefile with points
points_gdf = gpd.read_file('UK_DEM.shp')

# Check the coordinate reference systems (CRS)
print(f"Polygons CRS: {polygons_gdf.crs}")
print(f"Points CRS: {points_gdf.crs}")

# Ensure both GeoDataFrames have the same CRS
if polygons_gdf.crs != points_gdf.crs:
    points_gdf = points_gdf.to_crs(polygons_gdf.crs)
    print(f"Points reprojected to CRS: {points_gdf.crs}")

# Create an R-tree index for the bounding boxes
idx = rtree.index.Index()
for poly_id, geometry in enumerate(polygons_gdf.geometry):
    idx.insert(poly_id, geometry.bounds)

# Initialize a list to keep indices of points that are not within any polygon
remaining_indices = []

# Iterate through each point with a progress bar
for i, point in enumerate(tqdm(points_gdf.geometry, desc="Processing points")):
    # Get possible matching bounding boxes using the R-tree index
    possible_matches_index = list(idx.intersection(point.bounds))
    possible_matches = polygons_gdf.iloc[possible_matches_index]
    
    # Check if the point is actually within any of these polygons
    if not any(possible_matches.contains(point)):
        remaining_indices.append(i)

# Filter the points_gdf to keep only points not within any polygon
remaining_points_gdf = points_gdf.iloc[remaining_indices]

# Save the remaining points to a new shapefile
remaining_points_gdf.to_file('remaining_points.shp')

print(f"Filtered points saved successfully: remaining_points.shp")




reading start
reading over
Polygons CRS: EPSG:27700
Points CRS: EPSG:4326
Points reprojected to CRS: EPSG:27700


Processing points: 100%|██████████| 1346519/1346519 [29:38<00:00, 757.27it/s] 


Filtered points saved successfully: remaining_points.shp


Converting Vector to raster and removing 0 and no data value points 

In [58]:
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
import numpy as np

def vector_to_raster(vector_shapefile, reference_raster_file, output_raster_file, no_data_value=-9999):
    """
    Converts vector data from a shapefile to raster format using a reference raster.

    Parameters:
    - vector_shapefile: path to the vector shapefile (with columns: longitude, latitude, value)
    - reference_raster_file: path to the reference raster file
    - output_raster_file: path for the output raster file
    - no_data_value: value to represent no data in the raster
    """

    # Load the vector data
    vector_data = gpd.read_file(vector_shapefile)
    if 'longitude' not in vector_data.columns or 'latitude' not in vector_data.columns or 'value' not in vector_data.columns:
        raise ValueError("Shapefile must contain 'longitude', 'latitude', and 'value' columns.")

    # Open the reference raster
    with rasterio.open(reference_raster_file) as ref_raster:
        ref_transform = ref_raster.transform
        ref_crs = ref_raster.crs
        ref_width = ref_raster.width
        ref_height = ref_raster.height
        ref_pixel_size_x = ref_transform[0]
        ref_pixel_size_y = -ref_transform[4]  # Pixel size in y-direction is negative in geotransform
        ref_min_lon = ref_transform[2]
        ref_max_lat = ref_transform[5]

    # Create an empty raster array
    raster_data = np.full((ref_height, ref_width), no_data_value, dtype=np.float32)

    # Fill the raster data with values from vector data
    for _, row in vector_data.iterrows():
        lon = row['longitude']
        lat = row['latitude']
        value = row['value']
        x_idx = int((lon - ref_min_lon) / ref_pixel_size_x)
        y_idx = int((ref_max_lat - lat) / ref_pixel_size_y)
        if 0 <= x_idx < ref_width and 0 <= y_idx < ref_height:
            raster_data[y_idx, x_idx] = value

    # Create the output raster file
    with rasterio.open(
        output_raster_file, 'w', driver='GTiff',
        height=raster_data.shape[0], width=raster_data.shape[1],
        count=1, dtype=raster_data.dtype,
        crs=ref_crs, transform=ref_transform
    ) as dst:
        dst.write(raster_data, 1)
        dst.write_mask(raster_data != no_data_value)  # Set no-data mask

# Example usage
vector_shapefile = 'filtered_points_30m_DEM.shp'
reference_raster_file = 'UK_DEM2.tif'
output_raster_file = 'imputation_raster_30m.tif'

vector_to_raster(vector_shapefile, reference_raster_file, output_raster_file)


###  Resampling to the Temprature data resolution  

In [42]:
import os
import rasterio
from rasterio.enums import Resampling
import numpy as np
from tqdm import tqdm

# Define the resampling methods
resampling_methods = [
    Resampling.nearest,
    Resampling.bilinear,
    Resampling.cubic,
    Resampling.cubic_spline,
    Resampling.lanczos,
    Resampling.average,
    Resampling.mode,
    Resampling.max,
    Resampling.min,
    Resampling.med,
    Resampling.q1,
    Resampling.q3,
    Resampling.sum,
    Resampling.rms
]

# Paths to the input GeoTIFF files
raster1_path = 'UK_temp2.tif'  # This is the reference raster for resolution
raster2_path = 'imputation_raster_30m.tif'

# Main directory for storing the processed files
main_dir = 'manual_method_resampling'
os.makedirs(main_dir, exist_ok=True)

# List to store paths of all processed files
processed_file_paths = []

# Function to process the GeoTIFF to set zero values to NaN
def process_geotiff(data, profile, output_file):
    data[data == 0] = np.nan
    profile.update(dtype=rasterio.float32, nodata=np.nan)
    with rasterio.open(output_file, 'w', **profile) as dst:
        dst.write(data, 1)

# Loop through each resampling method with a progress bar
for method in tqdm(resampling_methods, desc="Resampling Methods"):
    method_name = method.name.lower()
    method_dir = os.path.join(main_dir, method_name)
    os.makedirs(method_dir, exist_ok=True)

    final_output_path = os.path.join(method_dir, f'UK_imputation_resampled_12km_{method_name}_processed.tif')

    # Step 1: Resample raster2 to match the resolution of raster1 using the current method
    with rasterio.open(raster1_path) as src_ref:
        profile_ref = src_ref.profile
        transform_ref = src_ref.transform

    with rasterio.open(raster2_path) as src:
        data = src.read(1)
        resampled_data = np.empty(src_ref.shape, dtype=np.float32)
        rasterio.warp.reproject(                                    # WARP reprojection 
            source=rasterio.band(src, 1),
            destination=resampled_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform_ref,
            dst_crs=profile_ref['crs'],
            resampling=method
        )

    # Step 2: Process the resampled data to set zero values to NaN and save the final file
    process_geotiff(resampled_data, profile_ref, final_output_path)
    print(f"The final raster with NaN values for zero data using {method_name} has been created and saved.")

    # Store the processed file path
    processed_file_paths.append(final_output_path)

# Print or use the list of processed file paths as needed
print("List of all processed file paths:")
for path in processed_file_paths:
    print(path)


Resampling Methods:   7%|▋         | 1/14 [00:01<00:16,  1.30s/it]

The final raster with NaN values for zero data using nearest has been created and saved.


Resampling Methods:  14%|█▍        | 2/14 [00:01<00:08,  1.35it/s]

The final raster with NaN values for zero data using bilinear has been created and saved.


Resampling Methods:  21%|██▏       | 3/14 [00:01<00:06,  1.81it/s]

...

The final raster with NaN values for zero data using q3 has been created and saved.


Resampling Methods:  93%|█████████▎| 13/14 [00:05<00:00,  2.30it/s]

The final raster with NaN values for zero data using sum has been created and saved.


Resampling Methods: 100%|██████████| 14/14 [00:06<00:00,  2.30it/s]

The final raster with NaN values for zero data using rms has been created and saved.
List of all processed file paths:
manual_method_resampling\nearest\UK_imputation_resampled_12km_nearest_processed.tif
manual_method_resampling\bilinear\UK_imputation_resampled_12km_bilinear_processed.tif
manual_method_resampling\cubic\UK_imputation_resampled_12km_cubic_processed.tif
manual_method_resampling\cubic_spline\UK_imputation_resampled_12km_cubic_spline_processed.tif
manual_method_resampling\lanczos\UK_imputation_resampled_12km_lanczos_processed.tif
manual_method_resampling\average\UK_imputation_resampled_12km_average_processed.tif
manual_method_resampling\mode\UK_imputation_resampled_12km_mode_processed.tif
manual_method_resampling\max\UK_imputation_resampled_12km_max_processed.tif
manual_method_resampling\min\UK_imputation_resampled_12km_min_processed.tif
manual_method_resampling\med\UK_imputation_resampled_12km_med_processed.tif
manual_method_resampling\q1\UK_imputation_resampled_12km_q1_pro

### Generate Temprature Data Points

In [43]:
import rasterio
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

# Path to your GeoTIFF file
input_file = 'UK_temp2.tif'

# Lists to store point geometries, coordinates, and pixel values
points = []
lats = []
lons = []
values = []

# Open the GeoTIFF file
with rasterio.open(input_file) as src:
    # Get the number of rows and columns in the raster
    rows, cols = src.shape
    
    # Get the nodata value
    nodata_value = src.nodata

    # Read the entire raster data into memory
    data = src.read(1)

    # Iterate through each pixel in the raster
    for row in range(rows):
        for col in range(cols):
            # Read pixel value
            pixel_value = data[row, col]
            
            # Check if pixel value is not nodata and not NaN
            if pixel_value != nodata_value and not np.isnan(pixel_value):
                # Convert pixel coordinates to geographic coordinates (longitude, latitude)
                lon, lat = src.xy(row, col)

                # Create a point geometry for each valid pixel coordinate
                point = Point(lon, lat)
                points.append(point)
                lats.append(lat)
                lons.append(lon)
                values.append(pixel_value)

# Create a GeoDataFrame from the list of points
gdf = gpd.GeoDataFrame(geometry=points, crs=src.crs)

# Add latitude, longitude, and pixel values to the GeoDataFrame
gdf['latitude'] = lats
gdf['longitude'] = lons
gdf['value'] = values

# Save the GeoDataFrame as a shapefile
output_shapefile = 'UK_temp.shp'
gdf.to_file(output_shapefile)

print(f"Shapefile saved successfully: {output_shapefile}")

Shapefile saved successfully: UK_temp.shp


### Converting the diffrence tiff to points and deleting extra points 
and removing 0 and no data value points 

In [47]:
import os
import rasterio
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
from tqdm import tqdm
import pandas as pd

# Function to convert raster to shapefile with points
def raster_to_points(input_file, output_shapefile):
    # Lists to store point geometries, coordinates, and pixel values
    points = []
    lats = []
    lons = []
    values = []

    # Open the GeoTIFF file
    with rasterio.open(input_file) as src:
        # Get the number of rows and columns in the raster
        rows, cols = src.shape
        
        # Get the nodata value
        nodata_value = src.nodata

        # Read the entire raster data into memory
        data = src.read(1)

        # Iterate through each pixel in the raster
        for row in tqdm(range(rows), desc="Converting Raster to Points"):
            for col in range(cols):
                # Read pixel value
                pixel_value = data[row, col]
                
                # Check if pixel value is not nodata and not NaN
                if pixel_value != nodata_value and not np.isnan(pixel_value):
                    # Convert pixel coordinates to geographic coordinates (longitude, latitude)
                    lon, lat = src.xy(row, col)

                    # Create a point geometry for each valid pixel coordinate
                    point = Point(lon, lat)
                    points.append(point)
                    lats.append(lat)
                    lons.append(lon)
                    values.append(pixel_value)

    # Create a GeoDataFrame from the list of points
    gdf = gpd.GeoDataFrame(geometry=points, crs=src.crs)

    # Add latitude, longitude, and pixel values to the GeoDataFrame
    gdf['latitude'] = lats
    gdf['longitude'] = lons
    gdf['value'] = values

    if not gdf.empty:
        # Save the GeoDataFrame as a shapefile if it is not empty
        gdf.to_file(output_shapefile)
        print(f"Shapefile saved successfully: {output_shapefile}")
    else:
        print(f"No valid points found in raster file: {input_file}")
    return gdf

# Function to filter out common points between two shapefiles
def filter_common_points(input_shapefile, reference_shapefile, output_shapefile):
    # Load the shapefiles
    gdf_input = gpd.read_file(input_shapefile)
    gdf_reference = gpd.read_file(reference_shapefile)

    # Ensure both GeoDataFrames have the same CRS
    if gdf_input.crs != gdf_reference.crs:
        gdf_reference = gdf_reference.to_crs(gdf_input.crs)

    # Identify points in gdf_input that are not in gdf_reference based on latitude and longitude
    gdf_filtered = gdf_input[~gdf_input.apply(lambda x: (x['latitude'], x['longitude']) in 
                                        set(gdf_reference.apply(lambda y: (y['latitude'], y['longitude']), axis=1)), axis=1)]

    if not gdf_filtered.empty:
        # Save the filtered GeoDataFrame to a new shapefile if it is not empty
        gdf_filtered.to_file(output_shapefile)
        print(f"Filtered shapefile saved successfully: {output_shapefile}")
    else:
        print(f"No points to save after filtering for file: {input_shapefile}")
    return gdf_filtered

# List of input raster files
input_raster_files = processed_file_paths

# Reference shapefile path
reference_shapefile = 'UK_temp.shp'

# List to store the results
results = []

# Loop through each input raster file
for input_raster_file in tqdm(input_raster_files, desc="Processing Files"):
    # Extract the directory and base name of the input file
    input_dir = os.path.dirname(input_raster_file)
    base_name = os.path.splitext(os.path.basename(input_raster_file))[0]
    
    # Define paths for intermediate and final shapefiles
    intermediate_shapefile = os.path.join(input_dir, f'{base_name}_intermediate.shp')
    final_output_shapefile = os.path.join(input_dir, f'{base_name}.shp')

    # Step 1: Convert raster to points and save as shapefile
    gdf_intermediate = raster_to_points(input_raster_file, intermediate_shapefile)

    if not gdf_intermediate.empty:
        # Step 2: Filter out common points based on reference shapefile and save the result
        gdf_filtered = filter_common_points(intermediate_shapefile, reference_shapefile, final_output_shapefile)

        if not gdf_filtered.empty:
            # Record the number of points in the final shapefile
            num_points = len(gdf_filtered)
            results.append({'File': final_output_shapefile, 'Num_Points': num_points})

    # Clean up the intermediate shapefile if needed
    if os.path.exists(intermediate_shapefile):
        os.remove(intermediate_shapefile)
        print(f"Intermediate shapefile removed: {intermediate_shapefile}")

# Convert the results to a DataFrame and save to a CSV file
if results:
    results_df = pd.DataFrame(results)
    results_csv_path = 'results_manual.csv'
    results_df.to_csv(results_csv_path, index=False)
    print(f"Results saved to CSV file: {results_csv_path}")
else:
    print("No results to save.")



Converting Raster to Points: 100%|██████████| 112/112 [00:00<00:00, 888.72it/s]


Shapefile saved successfully: manual_method_resampling\nearest\UK_imputation_resampled_12km_nearest_processed_intermediate.shp


Processing Files:   7%|▋         | 1/14 [00:02<00:29,  2.26s/it]

Filtered shapefile saved successfully: manual_method_resampling\nearest\UK_imputation_resampled_12km_nearest_processed.shp
Intermediate shapefile removed: manual_method_resampling\nearest\UK_imputation_resampled_12km_nearest_processed_intermediate.shp


Converting Raster to Points: 100%|██████████| 112/112 [00:00<00:00, 2754.85it/s]

...

Filtered shapefile saved successfully: manual_method_resampling\sum\UK_imputation_resampled_12km_sum_processed.shp
Intermediate shapefile removed: manual_method_resampling\sum\UK_imputation_resampled_12km_sum_processed_intermediate.shp


Converting Raster to Points: 100%|██████████| 112/112 [00:00<00:00, 803.76it/s]


Shapefile saved successfully: manual_method_resampling\rms\UK_imputation_resampled_12km_rms_processed_intermediate.shp


Processing Files: 100%|██████████| 14/14 [02:24<00:00, 10.30s/it]

Filtered shapefile saved successfully: manual_method_resampling\rms\UK_imputation_resampled_12km_rms_processed.shp
Intermediate shapefile removed: manual_method_resampling\rms\UK_imputation_resampled_12km_rms_processed_intermediate.shp
Results saved to CSV file: results_manual.csv


# Temportary Section (Strat remove current points to do imputation and move forward )

## Finding Temprature points in Costal region 